Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Source: Book Microservices Recipes
- - -

The example consists of three microservices: **Order**, **Customer** and **Catalog**.

**Order** uses **Catalog** and **Customer** with REST interface. In addition, each microservice offers some HTML pages.

In addition, an Apache web server is installed in the example, which allows the user to easily access the system with a website.

- - -

First we create the Kubernetes namespace ms-rest and label it so that Kiali can collect information.

In [ ]:
! kubectl create namespace ms-rest
! kubectl label  namespace ms-rest istio-injection=enabled

Now is a good time to launch the Kubernetes Dashboard and select the "ms-rest" namespace there.

Then we see how the next step is to create the microservices.

In [ ]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml
kubectl get service/apache --namespace ms-rest

Since we don't have a load balancer, we have to prepare the IP of the cluster and the mapped port (port-based-routing) as a URL using a small shell script.

In [ ]:
! [ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')

---
## Kiali - Observability


Kiali from the Istio project helps to understand the structure (= service mesh) of the application and its microservices and interfaces.

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

- - -
## Jaeger - Tracing

Jaeger is a system to track calls between microservices.

In [ ]:
! kubectl get service -n istio-system -l app=jaeger -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Jaeger  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/tracing -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Lasttest

In order to make the connections visible, we generate a little traffic.

In [ ]:
%%bash
URL="http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')"
for i in `seq 1 1000`;
do
    curl -s -o /dev/null -I -w "%{http_code}" ${URL}/order/order
    echo
done

- - -
### REST Paradigma

Representational State Transfer (abbreviated REST, more rarely also ReST) describes a programming paradigm for distributed systems.

REST is an abstraction of the structure and behavior of the World Wide Web (HTTP GET, PUT, POST, DELETE).

REST aims to create an architectural style that better represents the needs of the modern web. REST differs from other architectural styles primarily in the requirement for a uniform communication interface.

To illustrate the REST paradigm, we execute several API calls against our microservices using `curl` (Windows `Invoke-WebRequest`):

In [ ]:
%%bash
URL="http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')"
curl -s ${URL}/customer/customer | jq

In [ ]:
%%bash
URL="http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')"
curl -s ${URL}/catalog/catalog | jq

In [ ]:
%%bash
URL="http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')"
curl -s ${URL}/order/order/<Ersetzen mit order id> | jq

- - -

### Resilience

If the called microservice is not available at the moment, the caller must deal with the failure in such a way that it does not also fail.

That's why on Kubernetes, we can launch multiple instances of the dependent microservices!





In [ ]:
! kubectl --namespace ms-rest scale --replicas=2 deployment/customer
! kubectl --namespace ms-rest scale --replicas=2 deployment/order
! kubectl --namespace ms-rest scale --replicas=2 deployment/catalog
! kubectl --namespace ms-rest get pods -o wide

- - -

Clean up

In [ ]:
! kubectl delete namespace ms-rest

- - -
## Grafana

Grafana is an open-source, cross-platform application for graphing data from various data sources such as B. InfluxDB, MySQL, PostgreSQL, Prometheus and Graphite. The recorded raw data can then be output in various display formats

In [ ]:
! kubectl get service -n istio-system -l app.kubernetes.io/instance=grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Grafana  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app.kubernetes.io/instance=grafana -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')                                                         